In [31]:
import os
import os.path
import sys
import pandas as pd
import numpy as np
from importlib import reload

In [32]:
LOCAL_PATH = 'data'
TRACE_NAME = 'tencent_game_na'
RAW_TRACE = '{trace_name}.pcapng'.format(trace_name=TRACE_NAME)
TCP_TRACE_FEATURE_FILE = '{trace_name}_tcp_pkt.csv'.format(trace_name=TRACE_NAME)
UDP_TRACE_FEATURE_FILE = '{trace_name}_udp_pkt.csv'.format(trace_name=TRACE_NAME)
TCP_FLOW_FEATURE_FILE = '{trace_name}_tcp_flow.csv'.format(trace_name=TRACE_NAME)
UDP_FLOW_FEATURE_FILE = '{trace_name}_udp_flow.csv'.format(trace_name=TRACE_NAME)
# BUCKET_NAME = '' # replace with your bucket name
# KEY = '' # replace with your object key

In [33]:
if not os.path.exists(os.path.join(LOCAL_PATH, RAW_TRACE)):
    if not os.path.exists(LOCAL_PATH):
        os.mkdir(LOCAL_PATH)
    
    import boto3
    import botocore

    s3 = boto3.resource('s3')

    try:
        s3.Bucket(BUCKET_NAME).download_file(KEY, os.path.join(LOCAL_PATH, RAW_TRACE))
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist.")
        else:
            raise

In [34]:
# convert raw trace to readable udp and tcp packet feature csv file
from python import packet_feature
reload(packet_feature)
%time packet_feature.udp_generate(os.path.join(LOCAL_PATH, RAW_TRACE) , os.path.join(LOCAL_PATH, UDP_TRACE_FEATURE_FILE))
%time packet_feature.tcp_generate(os.path.join(LOCAL_PATH, RAW_TRACE) , os.path.join(LOCAL_PATH, TCP_TRACE_FEATURE_FILE))

Conversion done
CPU times: user 619 ms, sys: 60.3 ms, total: 679 ms
Wall time: 8.28 s
Conversion done
CPU times: user 1.02 s, sys: 76.9 ms, total: 1.1 s
Wall time: 9.27 s


In [35]:
def to_csv(trace_df, extract_func, file, features, max_packets_per_flow, time_delta_threshold):
    df = extract_func(trace_df, 1.0, upsampled=True, max_packets_per_flow=max_packets_per_flow, time_delta_threshold=time_delta_threshold)
    if df.shape[0] == 0:
        pd.DataFrame(columns=features).to_csv(file, index=False)
    else:
        df[features].to_csv(file, index=False)

In [36]:
from python import flow_feature
reload(flow_feature)

FEATURES = ['avg(pkt_len)', 'stddev(pkt_len)', 'fb_ratio', 'inter_arrival_time', 'pkt_count', 'duration']

tcp_trace_df = pd.read_csv(os.path.join(LOCAL_PATH, TCP_TRACE_FEATURE_FILE))
udp_trace_df = pd.read_csv(os.path.join(LOCAL_PATH, UDP_TRACE_FEATURE_FILE))
for trace_df in [tcp_trace_df, udp_trace_df]:
    if tcp_trace_df is trace_df:
        to_csv(trace_df, flow_feature.tcp_generate, os.path.join(LOCAL_PATH, TCP_FLOW_FEATURE_FILE), FEATURES, max_packets_per_flow=1000, time_delta_threshold=1)
    elif udp_trace_df is trace_df:
        to_csv(trace_df, flow_feature.udp_generate, os.path.join(LOCAL_PATH, UDP_FLOW_FEATURE_FILE), FEATURES, max_packets_per_flow=1000, time_delta_threshold=1)
    else:
        raise

tcp flow, flow pkt limit->1000, flow duration limit->1: 100%|██████████| 52936/52936 [00:11<00:00, 4745.66it/s]
/Users/shuojiang/Desktop/GA/Trace-Preprocessing/trace-feature-selection/python/flow_feature.py:82: RuntimeWarning: invalid value encountered in double_scalars
  flow['avg(pkt_len)'] = flow['pkt_len'] / flow['pkt_count']
/Users/shuojiang/anaconda2/envs/py36/lib/python3.6/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/Users/shuojiang/anaconda2/envs/py36/lib/python3.6/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/Users/shuojiang/anaconda2/envs/py36/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
udp flow, flow pkt limit->1000, flow duration limit->1: 100%|██████████| 43919/43919 [00:11<00:00, 38